# All the Imports

In [1]:
import sys
import os
import logging
sys.path.insert(1, '../src/')

In [2]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from lassonet import LassoNetClassifier


from RankCorr.picturedRocks import Rocks
import smashpy as smashpy


import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *


CPU times: user 9.83 s, sys: 528 ms, total: 10.4 s
Wall time: 10.4 s


In [3]:
from os import path

In [4]:
model_save_path = '../data/{}/models/'
num_times = 10

  

In [5]:
data_locations = [
                  'CITE-seq',
                 # 'developmental_mouse',
                  'mouse_brain_broad',
                  'paul',
                  'zeisel'
                 # 'synthetic'
]

In [6]:
result_axis = [
 ('All Markers', 'all_markers_results_{}.npy', 'all_markers_knn_results_{}.npy'), 
 ('L1 VAE', 'l1_vae_results_{}.npy', 'l1_vae_knn_results_{}.npy'),
 ('Smash RF', 'smash_rf_results_{}.npy', 'smash_rf_knn_results_{}.npy'),
 ('SMaSH DNN', 'smash_results_{}.npy', 'smash_knn_results_{}.npy'),
 ('RankCorr', 'rankcorr_results_{}.npy', 'rankcorr_knn_results_{}.npy'),
 ('Global Gate VAE', 'globalgate_vae_results_{}.npy', 'globalgate_vae_knn_results_{}.npy'),
 ('MarkerMap Unsupervised', 'marker_map_unsupervised_results_{}.npy', 'marker_map_unsupervised_knn_results_{}.npy'),
 ('MarkerMap Supervised', 'marker_map_supervised_results_{}.npy', 'marker_map_supervised_knn_results_{}.npy'),
 ('MarkerMap Mixed Supervision', 'marker_map_mixed_results_{}.npy', 'marker_map_mixed_knn_results_{}.npy'),
 ('Concrete VAE',  'concrete_vae_results_{}.npy', 'concrete_vae_knn_results_{}.npy'),
 ('LassoNet', 'lasso_net_results_{}.npy', 'lasso_net_knn_results_{}.npy')
]

## Mean Accuracies and F1

### Random Forest

In [7]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.076, 0.901)","(0.132, 0.83)","(0.064, 0.919)","(0.067, 0.917)","(0.066, 0.916)","(0.132, 0.83)","(0.091, 0.885)","(0.063, 0.924)","(0.065, 0.92)","(0.127, 0.836)","(0.062, 0.924)"
mouse_brain_broad,"(0.009, 0.991)","(0.163, 0.827)","(0.016, 0.984)","(0.013, 0.987)","(0.031, 0.969)","(0.089, 0.906)","(0.013, 0.987)","(0.006, 0.994)","(0.031, 0.969)","(0.126, 0.865)","(0.008, 0.991)"
paul,"(0.184, 0.796)","(0.531, 0.406)","(0.114, 0.883)","(0.17, 0.817)","(0.149, 0.845)","(0.382, 0.573)","(0.139, 0.856)","(0.108, 0.889)","(0.138, 0.857)","(0.398, 0.555)","(0.126, 0.868)"
zeisel,"(0.042, 0.957)","(0.192, 0.794)","(0.049, 0.951)","(0.086, 0.911)","(0.045, 0.954)","(0.187, 0.801)","(0.124, 0.868)","(0.045, 0.955)","(0.066, 0.931)","(0.2, 0.785)","(0.054, 0.945)"


### KNN

In [8]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    m, f1, _, _ = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 3)
    f1 = round(f1, 3)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.098, 0.898)","(0.191, 0.782)","(0.071, 0.912)","(0.066, 0.925)","(0.073, 0.913)","(0.18, 0.796)","(0.104, 0.885)","(0.062, 0.929)","(0.07, 0.92)","(0.18, 0.795)","(0.064, 0.926)"
mouse_brain_broad,"(0.103, 0.885)","(0.234, 0.744)","(0.027, 0.973)","(0.013, 0.987)","(0.041, 0.958)","(0.14, 0.848)","(0.015, 0.985)","(0.006, 0.994)","(0.041, 0.958)","(0.198, 0.782)","(0.009, 0.991)"
paul,"(0.321, 0.608)","(0.632, 0.375)","(0.217, 0.768)","(0.291, 0.676)","(0.257, 0.729)","(0.447, 0.521)","(0.268, 0.721)","(0.203, 0.789)","(0.253, 0.735)","(0.461, 0.513)","(0.228, 0.761)"
zeisel,"(0.054, 0.945)","(0.261, 0.726)","(0.047, 0.953)","(0.103, 0.892)","(0.047, 0.953)","(0.273, 0.717)","(0.153, 0.84)","(0.038, 0.961)","(0.067, 0.93)","(0.286, 0.7)","(0.059, 0.938)"


## Variances

### Random Forest

In [9]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[1], num_times)
    m = round(m, 15)
    f1 = round(f1, 15)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results1 = pd.DataFrame(results)
tabulated_results1

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.006872020055508, 0.010136166651257)","(0.022163617868238, 0.029416864610836)","(0.003213373314261, 0.00555966722691)","(0.00687789274143, 0.010105816083535)","(0.004733690432275, 0.006798392787681)","(0.023405501304022, 0.028250144398124)","(0.013660677767388, 0.018455912101701)","(0.005746852007657, 0.007687809743615)","(0.004614559884733, 0.006506470242331)","(0.017066085032524, 0.022632272082965)","(0.007748343145938, 0.010735006527034)"
mouse_brain_broad,"(0.001181458750686, 0.001250291767824)","(0.033728067598674, 0.03731277900943)","(0.001364153088291, 0.001306289357603)","(0.0, 0.0)","(0.002867581535675, 0.002862101977053)","(0.03134134418024, 0.033609654947977)","(0.003996364301167, 0.004190985626791)","(0.000652549771534, 0.000692909758018)","(0.014578444654082, 0.01495887722167)","(0.028334411856361, 0.030410217150315)","(0.001031501143525, 0.001119587161809)"
paul,"(0.013973530813342, 0.016251156085517)","(0.0328901219247, 0.030680873053957)","(0.017297756707865, 0.017107260105196)","(0.026743062689314, 0.031115240877233)","(0.011254426004203, 0.012610258067416)","(0.029647709914301, 0.03471944494339)","(0.014570515093919, 0.015843098132339)","(0.012216268238573, 0.01231799454935)","(0.021302972448354, 0.021007726378302)","(0.02604083895557, 0.034662299056728)","(0.008989297576416, 0.010087901889253)"
zeisel,"(0.006990333290383, 0.007214134999314)","(0.035409671167187, 0.040280393656022)","(0.005558500195837, 0.005835543017565)","(0.021723211510584, 0.023752071686278)","(0.0051446338881, 0.005226262998764)","(0.045678043992489, 0.049285721054077)","(0.048104654907401, 0.05401823739169)","(0.006261193917546, 0.006478326264052)","(0.010784546038668, 0.01218630903284)","(0.044230783929963, 0.051432224503959)","(0.009552545819905, 0.009544864725478)"


### KNN

In [10]:
results = {}
for e, axis in itertools.product(data_locations, result_axis):
    _, _, m, f1 = model_variances(model_save_path.format(e) + axis[2], num_times)
    m = round(m, 15)
    f1 = round(f1, 15)
    if axis[0] in results:
        results[axis[0]][e] = (m, f1)
    else:
        results[axis[0]] = {}
        results[axis[0]][e] = (m, f1)

tabulated_results2 = pd.DataFrame(results)
tabulated_results2

,All Markers,L1 VAE,Smash RF,SMaSH DNN,RankCorr,Global Gate VAE,MarkerMap Unsupervised,MarkerMap Supervised,MarkerMap Mixed Supervision,Concrete VAE,LassoNet
CITE-seq,"(0.00539910732168, 0.005169739098054)","(0.035480685120017, 0.041715519636621)","(0.0, 0.0)","(0.0, 0.0)","(0.004197227961746, 0.005731363246282)","(0.028446175848583, 0.031686438079081)","(0.016218495758032, 0.017209641575839)","(0.004181165032701, 0.005023243442435)","(0.004938576532769, 0.006365682182415)","(0.023597339976126, 0.028182020369063)","(0.0052666169201, 0.007347726268675)"
mouse_brain_broad,"(0.006449210299109, 0.006078219914174)","(0.046261663131635, 0.051471007921292)","(0.000113679424024, 9.7971021558e-05)","(0.0, 0.0)","(0.003089549992866, 0.003059013715177)","(0.051279265220486, 0.059101703833484)","(0.007142575144417, 0.007414663177989)","(0.000772458499503, 0.000771996192869)","(0.021229991045893, 0.02166159923359)","(0.03768919263757, 0.043405916281869)","(0.001271978429231, 0.001321228096279)"
paul,"(0.016805962999201, 0.022552471198598)","(0.058924568884793, 0.051463852289275)","(0.015949846928262, 0.019118963371908)","(0.018479118323267, 0.023532043605783)","(0.030199793590464, 0.036888726295609)","(0.024455179213481, 0.028860569676006)","(0.014748992252958, 0.013765671748963)","(0.017345202293842, 0.01849414855667)","(0.031699270529407, 0.033154947391888)","(0.034313664603573, 0.035680588292003)","(0.022161172161172, 0.024821649845612)"
zeisel,"(0.00825935018036, 0.008494210005314)","(0.035131802366684, 0.036415671495428)","(0.004991680532446, 0.005121891999959)","(0.041907661102749, 0.045803267713357)","(0.006357728177884, 0.006356651137681)","(0.056097892729881, 0.061257961883791)","(0.04963871510874, 0.05202863732319)","(0.004553721192481, 0.004651296218417)","(0.0119765663314, 0.012631493984938)","(0.044263633106143, 0.049575326771285)","(0.01007308543605, 0.01166240504249)"


In [11]:
indices =  np.load('../data/paul/models/experiment_data_folds/rankcorr_indices_7.npy', allow_pickle = True)

In [12]:
indices2 =  np.load('../data/paul/models/experiment_data_folds/rankcorr_indices_9.npy', allow_pickle = True)

In [13]:
indices[1]

array([1751, 2032,  173, 1158, 1663,  533, 1729,  885, 1905, 2699,  416,
       1839, 2566, 1571, 1097, 1154, 1047, 1005, 1754,  627, 1876, 1741,
       1730, 1561, 2612, 2479,  624, 2163, 1931, 2440, 1982, 2480,  558,
        855, 2188, 1622, 1245,  595,  439, 1844,   79,  930, 2587,  677,
         23, 1643,  579,  198,  577,  428, 1667, 1819, 2017, 1393, 2079,
       1020,  180,  459,  905,  258, 1765,  137, 1888, 2598,  255, 2646,
        316, 1426, 1786,  494,  168,  409, 1866, 1478,  141, 2097, 1830,
        860, 1572, 1129, 1959,  451,  985, 1030,  727, 1809, 1467, 1556,
       1411,  181,  593,  567,  656, 1666, 1384, 1938, 1070,  804, 2293,
        842, 1543,  103, 1328, 2285, 2513, 1604, 2243, 1108,  776, 1305,
        993, 1173,  474,  851, 2642, 1948,   69,  583, 2178,  136, 1976,
       2469, 1937, 2115,  499, 1167,  726,  426, 1671, 1325, 2013,  616,
        862,  476, 1246,  587,  867,   97, 1763, 1816,  105,  538,  442,
       2388,  259,  981, 2537, 1947,  223, 1335,  4

In [14]:
indices2[1]

array([1334, 1671, 1732, 1731,  319, 1628, 1454, 2238,  705, 2701, 2590,
       1764,  409,  190, 2416,  139,  788,  532,  376, 2167,  327,  731,
        184, 2456,  972,  445, 1751, 1462,  338,  758, 1436,  402, 1358,
       2223, 1949,  545, 2243,   99, 1490, 1526,  387, 1554, 1267,   55,
       2002,  217,  760, 2654,  122, 2267,  369,  548, 2696,   96,  618,
       1623, 2001, 2351, 2211, 1326, 1698,  841, 2270, 2685, 2435, 1709,
       1119, 1052,  490, 1003,  469, 1532,  602, 2275, 1721,  293, 2304,
       1097, 2066,  347, 2645, 2195,  916, 1865,  992, 2252,  709, 2710,
       1163,  875, 1631, 1496, 2004,  118,  104, 2301,  806, 2150, 2526,
        579, 2334,  198, 1683,  479,  999,  230,  703, 2431,    0, 2535,
       1593, 1627,  755, 1614, 2200, 2465, 1223, 1611,  236, 2681,  923,
        518,  413, 1202, 1150, 2119, 1643, 2689, 2288,  983, 1854,  838,
       2643, 2302, 2594, 1352,  263, 1482,  836, 2309, 1968, 2023,  901,
       1941, 2621,  813, 1752, 2380, 1652,  711, 26

In [15]:
from os import path
dataset_dir = '../data/zeisel/'
model_save_path = '../data/zeisel/models/'
viz_save_path = '../data/zeisel/visualizations/'

if not path.exists(model_save_path):
  os.makedirs(model_save_path)

if not path.exists(model_save_path + 'experiment_data_folds/'):
  os.makedirs(model_save_path + 'experiment_data_folds/')
  
if not path.exists(viz_save_path):
  os.makedirs(viz_save_path)

adata = sc.read_h5ad(dataset_dir + "Zeisel.h5ad")
X = adata.X.copy()
adata.obs['names']=adata.obs['names0']
adata.obs['annotation'] = adata.obs['names0']
labels = adata.obs['names0'].values
encoder = LabelEncoder()
encoder.fit(labels)
y = encoder.transform(labels)

input_size = X.shape[1]
y

array([2, 2, 2, ..., 1, 1, 1])

In [16]:
_, _, _, train_indices, val_indices, test_indices = split_data_into_dataloaders(X, y, 0.7, 0.1)
train_X_y = adata[np.concatenate([train_indices, val_indices]), :]    

markers = np.load('../data/zeisel/models/all_markers_1.npy', allow_pickle=True)
model = K
train_x = X[np.concatenate([train_indices, val_indices]), :]
train_y = y[np.concatenate([train_indices, val_indices])]
test_x = X[test_indices,:]
test_y = y[test_indices]
    
if markers is not None:
    train_x = train_x[:, markers]
    test_x = test_x[:, markers]

if model is None:
    model = RandomForestClassifier()
model.fit(train_x, train_y)
pred_y = model.predict(test_x)
train_rep = classification_report(train_y, model.predict(train_x), output_dict=True)
test_rep = classification_report(test_y, pred_y, output_dict=True)
cm = confusion_matrix(test_y, pred_y)
if cm is None:
    raise Exception("Some error in generating confusion matrix")
misclass_rate = 1 - accuracy_score(test_y, pred_y)


NameError: name 'K' is not defined

In [ ]:
train_indices

In [ ]:
np.load('../data/zeisel/models/all_markers_1.npy', allow_pickle=True)